# Internacoes count data
All data was extracted from the SIH database

Import libraries

In [57]:
import pandas as pd
import glob
root = '../'

Convert raw data to formatted count data

In [37]:
path = root + 'CSV/TabNet/Internacoes/'
all_files = glob.glob(path + "*.csv")
for file in all_files:
    file_name = file.split("\\")[1]
    disease = file_name.split('.csv')[0]
    disease_df = pd.read_csv(path + file_name, sep=';')
    disease_df['MUNCOD'] = disease_df['Município'].str.split(' ', expand=True)[0]
    disease_df = disease_df[disease_df["Município"] != "Total"]
    disease_df.drop(['Município', 'Total'], axis=1, inplace=True)
    disease_df = disease_df.replace("-", 0)
    disease_df = disease_df.astype(int)
    for col in disease_df.columns:
        if(col != "MUNCOD"):
            year = col.split("/")[0]
            disease_df[year] = np.zeros(disease_df.shape[0])
            disease_df[year] = disease_df[year] + disease_df[col]
            disease_df = disease_df.drop(columns=col)
    disease_df.to_csv(root + 'CSV/TabNet/Internacoes_Anual/' + disease +'.csv')

# Internacoes rate data

In [58]:
def drop_if_exists(col_list, df):
    for col in col_list:
        if col in df.columns:
            df = df.drop(columns=col, axis=1)
    return df

In [59]:
path = root + 'CSV/TabNet/Internacoes_Anual/'
all_files = glob.glob(path + "*.csv")
population = pd.read_csv(root + 'CSV/Population/population_08_18.csv', index_col=[0])
for file in all_files:
    file_name = file.split("\\")[1]
    disease = file_name.split(".csv")[0]
    disease_df = pd.read_csv(path + file_name,index_col=0)
    disease_df = drop_if_exists(["2007","2019","2020"],disease_df)
    df = pd.merge(population, disease_df, left_on="MUNCOD", right_on="MUNCOD")
    for column in population.columns:
        if column.split('_')[0] == "POP":
            year = column.split('_')[1]
            if '20' + year in df.columns:
                df['RATE_' + year] = df['20' + year]/df['POP_' + year] * 100000
            df = drop_if_exists(['20' + year, 'POP_' + year], df)
    df.to_csv(root + 'CSV/TabNet/Internacoes_Rate/' + disease + ".csv")
df.head()

,MUNCOD,RATE_18,RATE_17,RATE_16,RATE_15,RATE_14,RATE_13,RATE_12,RATE_11,RATE_10,RATE_09,RATE_08
0,110001,0.000000,3.931281,0.0,0.000000,0.000000,3.886816,4.154722,8.254912,4.094669,4.106102,0.0
1,110002,0.000000,0.000000,0.0,2.873536,0.972195,0.000000,0.000000,3.276182,1.106758,1.169030,0.0
2,110003,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,110004,2.358129,0.000000,0.0,1.146447,2.310643,2.329292,2.521114,0.000000,0.000000,0.000000,0.0
4,110005,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
